In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data129148  dataset  readme.json


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 3.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [5]:
#导入需要的包
import os
import zipfile
import random
import json
import cv2
import numpy as np
from PIL import Image
import paddle
import matplotlib.pyplot as plt
from paddle.io import Dataset


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [6]:
'''
参数配置
'''
train_parameters = {
    "input_size": [16,224,224],                           #输入图片的shape
    "class_dim": -1,                                     #分类数
    "src_path":"data/data129148/sports-classification.zip",       #原始数据集路径
    "target_path":"/home/aistudio/data/dataset",        #要解压的路径 
    "train_list_path": "./train.txt",              #train_data.txt路径
    "eval_list_path": "./eval.txt",                  #eval_data.txt路径
    "label_list_path": "./label.txt",                  #eval_data.txt路径
    "label_dict":{},                                    #标签字典
    "readme_path": "/home/aistudio/data/readme.json",   #readme.json路径
    "num_epochs": 40,                                    #训练轮数
    "train_batch_size": 16,                             #批次的大小
    "learning_strategy": {                              #优化函数相关的配置
        "lr": 0.001                                     #超参数学习率
    } 
}




In [7]:
def unzip_data(src_path,target_path):

    '''
    解压原始数据集，将src_path路径下的zip包解压至data/dataset目录下
    '''

    if(not os.path.isdir(target_path)):    
        z = zipfile.ZipFile(src_path, 'r')
        z.extractall(path=target_path)
        z.close()
    else:
        print("文件已解压")



In [8]:
def get_data_list(target_path,train_list_path,eval_list_path,label_list_path):
    '''
    生成数据列表
    '''
    #存放所有类别的信息
    class_detail = []
    #获取所有类别保存的文件夹名称
    data_list_path=target_path
    roots = os.listdir(data_list_path)
    if '__MACOSX' in roots:
        roots.remove('__MACOSX')
    # #总的图像数量
    all_class_images = 0
    # #存放类别标签
    class_label=0
    # #存放类别数目
    class_dim = 0
    # #存储要写进eval.txt和train.txt中的内容
    trainer_list=[]
    eval_list=[]
    label_list=[]
    #读取每个类别
    for root in roots:
        if root!='train':
            continue;
        temp1=os.path.join(data_list_path,root)
        class_dirs=os.listdir(temp1)
        for class_dir in class_dirs:
            if class_dir != ".DS_Store":
                while ' ' in class_dir:    #若文件夹名中存在空格
                    d=class_dir.split(' ')    #以空格作为分隔符
                    for i in range(len(d)-1):
                        new_name1=os.path.join(d[i]+"_"+d[i+1])
                    os.rename(os.path.join(temp1,class_dir),os.path.join(temp1,new_name1))  #重命名 
                    class_dir=new_name1           
                class_dim += 1
                #每个类别的信息
                class_detail_list = {}
                eval_sum = 0
                trainer_sum = 0
                #统计每个类别有多少张图片
                class_sum = 0
                #获取类别路径 
                path = os.path.join(temp1,class_dir)
                # 获取所有图片
                img_paths = os.listdir(path)
                for img_path in img_paths:                                  # 遍历文件夹下的每个图片
                    if img_path =='.DS_Store':
                        continue
                    name_path = os.path.join(path,img_path)                       # 每张图片的路径
                    if class_sum % 15 == 0:                                 # 每10张图片取一个做验证数据
                        eval_sum += 1                                       # eval_sum为测试数据的数目
                        eval_list.append(name_path + "\t%d" % class_label + "\n")
                    else:
                        trainer_sum += 1 
                        trainer_list.append(name_path + "\t%d" % class_label + "\n")#trainer_sum测试数据的数目
                    class_sum += 1                                          #每类图片的数目
                    all_class_images += 1                                   #所有类图片的数目         
                # 说明的json文件的class_detail数据
                class_detail_list['class_name'] = class_dir             #类别名称
                class_detail_list['class_label'] = class_label          #类别标签
                class_detail_list['class_eval_images'] = eval_sum       #该类数据的测试集数目
                class_detail_list['class_trainer_images'] = trainer_sum #该类数据的训练集数目
                class_detail.append(class_detail_list)  
                #初始化标签列表
                train_parameters['label_dict'][str(class_label)] = class_dir
                #label_dict字典中，类别标签为class_
                class_label += 1
                label_list.append(class_dir+"\n")
            
            
    #初始化分类数
    train_parameters['class_dim'] = class_dim
    print(train_parameters)
    #乱序  
    random.shuffle(eval_list)
    with open(eval_list_path, 'a') as f:
        for eval_image in eval_list:
            f.write(eval_image) 
    #乱序        
    random.shuffle(trainer_list) 
    with open(train_list_path, 'a') as f2:
        for train_image in trainer_list:
            f2.write(train_image) 
   
    with open(label_list_path, 'a') as f2:
        i=0
        for image_label in label_list:
            f2.write('{} {}'.format(i,image_label))
            i=i+1 
    # 说明的json文件信息
    readjson = {}
    readjson['all_class_name'] = data_list_path                  #文件父目录
    readjson['all_class_images'] = all_class_images
    readjson['class_detail'] = class_detail
    jsons = json.dumps(readjson, sort_keys=True, indent=4, separators=(',', ': '))
    with open(train_parameters['readme_path'],'w') as f:
        f.write(jsons)
    print ('生成数据列表完成！')


In [9]:
'''
参数初始化
'''
src_path=train_parameters['src_path']
target_path=train_parameters['target_path']
train_list_path=train_parameters['train_list_path']
eval_list_path=train_parameters['eval_list_path']
label_list_path=train_parameters['label_list_path']
batch_size=train_parameters['train_batch_size']
'''
解压原始数据到指定路径
'''
unzip_data(src_path,target_path)




文件已解压


In [10]:
'''
划分训练集与验证集，乱序，生成数据列表
'''
#每次生成数据列表前，首先清空train.txt和eval.txt
with open(train_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
with open(eval_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
with open(label_list_path, 'w') as f: 
    f.seek(0)
    f.truncate()     

target_path='/home/aistudio/data/dataset/sports-classification'
#生成数据列表   
get_data_list(target_path,train_list_path,eval_list_path,label_list_path)

{'input_size': [16, 224, 224], 'class_dim': 100, 'src_path': 'data/data129148/sports-classification.zip', 'target_path': '/home/aistudio/data/dataset', 'train_list_path': './train.txt', 'eval_list_path': './eval.txt', 'label_list_path': './label.txt', 'label_dict': {'0': 'jousting', '1': 'ice_climbing', '2': 'hurdles', '3': 'arm_wrestling', '4': 'air_hockey', '5': 'boxing', '6': 'hang_gliding', '7': 'polo', '8': 'balance_beam', '9': 'cheerleading', '10': 'water_polo', '11': 'gaga', '12': 'field_hockey', '13': 'javelin', '14': 'basketball', '15': 'pole_climbing', '16': 'barell_racing', '17': 'motorcycle_racing', '18': 'cricket', '19': 'baton_twirling', '20': 'hammer_throw', '21': 'bull_riding', '22': 'horse_racing', '23': 'fencing', '24': 'table_tennis', '25': 'sumo_wrestling', '26': 'tennis', '27': 'water_cycling', '28': 'shot_put', '29': 'olympic_wrestling', '30': 'wheelchair_basketball', '31': 'high_jump', '32': 'weightlifting', '33': 'golf', '34': 'parallel_bar', '35': 'curling', '3

In [11]:
!pip install paddlex==2.0.0


Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
import paddlex as pdx
from paddlex import transforms as T
train_transforms = T.Compose(
    [T.RandomCrop(crop_size=224), T.RandomHorizontalFlip(), T.Normalize()])

eval_transforms = T.Compose([
    T.ResizeByShort(short_size=256), T.CenterCrop(crop_size=224), T.Normalize()
])


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:171: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[05-13 22:36:39 MainThread @utils.py:79] WRN paddlepaddle version: 2.4.0. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: FutureWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddleslim/common/analyze_helper.py:22: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://matplotlib_inline.backend_inline' by the following code:
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(cod

In [13]:
train_dataset = pdx.datasets.ImageNet(
    data_dir='',
    file_list='/home/aistudio/train.txt',
    label_list='/home/aistudio/label.txt',
    transforms=train_transforms,
    shuffle=True)
    
eval_dataset = pdx.datasets.ImageNet(
    data_dir='',
    file_list='/home/aistudio/eval.txt',
    label_list='/home/aistudio/label.txt',
    transforms=eval_transforms)

num_classes = len(train_dataset.labels)
model = pdx.cls.ResNet101_vd_ssld(num_classes=num_classes)
model.train(num_epochs=15,
            train_dataset=train_dataset,
            train_batch_size=16,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[6, 8],
            save_interval_epochs=1,
            learning_rate=0.001,
            save_dir='output/resNet101_vd_ssld',
            use_vdl=True)


2024-05-13 22:36:42 [INFO]	Starting to read file list from dataset...
2024-05-13 22:36:42 [INFO]	12623 samples in file /home/aistudio/train.txt
2024-05-13 22:36:42 [INFO]	Starting to read file list from dataset...
2024-05-13 22:36:42 [INFO]	949 samples in file /home/aistudio/eval.txt
2024-05-13 22:36:42 [INFO]	Loading pretrained model from output/resNet101_vd_ssld/pretrain/ResNet101_vd_ssld_pretrained.pdparams


W0513 22:36:42.382019  1046 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.2
W0513 22:36:42.386389  1046 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


In [21]:
import paddlex as pdx
model = pdx.load_model('output/resNet101_vd_ssld/best_model')
image_name = '/home/aistudio/data/dataset/sports-classification/valid/baseball/1.jpg'
result = model.predict(image_name)
print(result)

2024-05-13 23:12:02 [INFO]	Model[ResNet101_vd_ssld] loaded.
[{'category_id': 37, 'category': '37 baseball', 'score': 0.99730444}]
